## Import libraries 

In [2]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
import re 

Read Excel with Pandas library 

In [3]:
df = pd.read_excel("datasets/build/wtj/data_labeled_wtj.xlsx")
df.head()

,subject,body,from,date,label
0,"Julien, candidature bien reçue pour Data Engineer","↥ Reply above this line ↥\n\nBonjour Julien,\...",as-a038ff5c9da37cda54eab016@candidates.welcome...,2025-12-26 15:33:01,1
1,"Julien, candidature bien reçue pour Consultant...",↥ Répondre au dessus de cette ligne ↥\n\nBonj...,kanbi-d47fd7b8ad633c7d0013c911@candidates.welc...,2026-01-01 21:00:00,1
2,"Julien, candidature bien reçue pour Data Engin...",↥ Répondre au dessus de cette ligne ↥\n\nBonj...,homee-ab1cf5751dc5827457722af0@candidates.welc...,2025-08-26 12:07:00,1
3,"Julien, candidature bien reçue pour Consultant...",↥ Répondre au dessus de cette ligne ↥\n\nBonj...,mc-f85ce6a15575d2785996c40d@candidates.welcome...,2025-08-26 12:27:00,1
4,"Julien, candidature bien reçue pour Data Analy...",↥ Répondre au dessus de cette ligne ↥\n\nBonj...,homee-40161024241f63fbc5dbca6b@candidates.welc...,2025-08-26 12:39:00,1


## Basics EDA (appropriate in this context)

### 1. Analyzing the data  

In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   subject  49 non-null     object        
 1   body     49 non-null     object        
 2   from     49 non-null     object        
 3   date     49 non-null     datetime64[ns]
 4   label    49 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.0+ KB


### 2. Checking missing values 

In [5]:
df.isnull().sum()

subject    0
body       0
from       0
date       0
label      0
dtype: int64

### 3. Checking for the duplicates values 

In [6]:
df.nunique()

subject    43
body       47
from       36
date       47
label       4
dtype: int64

We already know with the Linkedin analyse that there is pattern on the mail ... let's verify if we can't deduct something about the wtj dataset 

In [7]:
print(*set(df["from"])) # Set = drop duplicates, * = unpack the set 

actin-d42b5f3b645bfc3f7dd0afc1@candidates.welcomekit.co quide-edb2fbd16af63343e0d19f1a@candidates.welcomekit.co ekie-f478adfc16502cefc04d24e7@candidates.welcomekit.co  polyn-731bb1643c9e7ea55a710061@candidates.welcomekit.co dacke-a8b2edfa31083bbaa75a1db9@candidates.welcomekit.co tcst-054dca9017801b0260f6619b@candidates.welcomekit.co urgo-bb2eb99d52f5310b7d51ebc5@candidates.welcomekit.co til-fe2915fcb8a6fbf8b3f4c9b4@candidates.welcomekit.co ika-dd37fa6ff07649e9e50cc6b4@candidates.welcomekit.co arkhn-41acf78d451d889ced56fa85@candidates.welcomekit.co ca-ccd633d8df3969a4a4af6853@candidates.welcomekit.co kanbi-d47fd7b8ad633c7d0013c911@candidates.welcomekit.co erida-a0b5c0fa2c736dd29b845760@candidates.welcomekit.co upply-88e36369d3eb5f1dddf7d121@candidates.welcomekit.co di-cd243b0776247d61302c5ca5@candidates.welcomekit.co ekie-b6a779a688631411de447401@candidates.welcomekit.co skill-5d7f554e25736b4d1bdabe04@candidates.welcomekit.co 79-66e612167546c2759c4b5844@candidates.welcomekit.co eduma-0b

In [8]:
import re

emails = list(df["from"])
domains = [re.search(r"@.*", e).group().strip() for e in emails] # strip pour supprimer espace début/fin 
print(*set(domains))

@candidates.welcomekit.co


<b>FIRST RULE : VERIFY THAT THE MAIL IS ENDING BY @candidates.welcomekit.com TO ANALYSE IT</b>

## I wanna see for each label, which words is the most common in the subject :  

In [9]:
print(set(df["label"])) # all the label values

{1, 2, 3, 4}


In [10]:
def get_words_by_label_in_subject(df, label_value):
    return (
        df.loc[df["label"] == label_value, "subject"]
        .str.lower()
        .str.split()
        .explode()
    )

### number element labelised 1

In [11]:
nb_label0 = len(list(df.loc[df["label"] ==1, "subject"]))
print(nb_label0)

26


In [12]:
words = get_words_by_label_in_subject(df,1)
words.value_counts().head(10)

candidature    25
pour           23
julien,        21
bien           20
reçue          19
data           17
engineer       14
-              10
(h/f)           6
/               5
Name: subject, dtype: int64

In [13]:
df.loc[
    (df["label"] == 1) &
    (~df["subject"].str.contains("candidature", case=False)), # case = ignore majuscule/minuscule, ~ = négation
    "subject"
]

18    Julien, thanks for your application for Data E...
Name: subject, dtype: object

In [14]:
df.loc[
    (df["label"] == 1) &
    (~df["subject"].str.contains("pour", case=False)), # case = ignore majuscule/minuscule, ~ = négation
    "subject"
]

7     ARMIS - Candidature Solutions Engineer - (CDI)...
13        Ta candidature est bien arrivée chez Yomoni 🚀
18    Julien, thanks for your application for Data E...
Name: subject, dtype: object

### THINKING : WHEN THE APPLICATION IS IN FRENCH AND CONTAIN "candidature" IT COULD MEANS THAT THE COMPANY JUST RECEIVE YOUR APPLICATION   

### number element labelised 2

In [15]:
nb_label2 = len(list(df.loc[df["label"] ==2, "subject"]))


18


In [17]:
words = get_words_by_label_in_subject(df,2)

words.value_counts().head()

candidature    18
-              14
de             13
poste          11
votre          11
Name: subject, dtype: int64

### THINKING : WHEN THE APPLICATION IS IN FRENCH AND CONTAIN "candidature" IT COULD MEANS THAT THE COMPANY JUST DECLINED YOUR APPLICATION

### number element labelised 4 (rule defined ✅)

In [84]:
nb_label4 = len(list(df.loc[df["label"] ==4, "subject"]))
print(nb_label4)

12


In [85]:
words = get_words_by_label_in_subject(df,4)

words.value_counts()[lambda x: x == nb_label4]

votre          12
a              12
été            12
vue            12
par            12
candidature    12
Name: subject, dtype: int64

In [88]:
df.loc[df["label"] ==4, "subject"].head()

93         Votre candidature a été vue par XRAYS TRADING
100                Votre candidature a été vue par ALTEN
128                Votre candidature a été vue par Extia
130            Votre candidature a été vue par MR SEARCH
132    Votre candidature a été vue par DGSE - Directi...
Name: subject, dtype: object

The last results let us understand that the pattern : <b>"votre candidature a été vue par"</b> appear for each element labelised 4 (see the word document to remind the labels meaning)

<b> THIRD RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "votre candidature a été vue par" IT MEANS THAT THE COMPANY JUST SAW YOUR APPLICATION (POSITIVE WITHOUT ACTION) </b>

### number labelised 5 (rule defined ✅)

In [86]:
nb_label5 = len(list(df.loc[df["label"] ==5, "subject"]))
print(nb_label5)

23


In [87]:
words = get_words_by_label_in_subject(df,5)

words.value_counts()[lambda x: x == nb_label5]

votre          23
:              23
chez           23
candidature    23
Name: subject, dtype: int64

In [89]:
df.loc[df["label"] ==5, "subject"].head()

92     Votre candidature : Développeur SQL/Développeu...
95     Votre candidature : Data Engineer H/F chez Gro...
96           Votre candidature : Data Engineer chez CBTW
99     Votre candidature : Ingénieur Data PL/SQL - Co...
101    Votre candidature : Data engineer (F/H) chez B...
Name: subject, dtype: object

The last results let us understand that the pattern : <b>"Votre candidature : POSTE chez"</b> appear for each element labelised 5 (see the word document to remind the labels meaning)

<b> FOURTH RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Votre candidature : [...] chez " IT MEANS THAT THE COMPANY DECLINED YOUR APPLICATION </b>

### REVIEW (ALL RULES) 

<b>FIRST RULE : VERIFY THAT THE MAIL IS COMING FROM jobs-noreply@linkedin.com TO ANALYSE IT</b> <br>

<b> SECOND RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Name, votre candidature a été envoyée à" IT MEANS THAT THE COMPANY JUST RECEIVE THE APPLICATION </b> <br>

<b> THIRD RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "votre candidature a été vue par" IT MEANS THAT THE COMPANY JUST SAW YOUR APPLICATION (POSITIVE WITHOUT ACTION) </b> <br>

<b> FOURTH RULE : VERIFY HOW IS STRUCTURED THE SUBJECT MESSAGE, IF -> "Votre candidature : [...] chez " IT MEANS THAT THE COMPANY DECLINED YOUR APPLICATION </b>